# Download data and unzip

In [ ]:
!wget --content-disposition 'https://docs.google.com/uc?export=download&id=1-mpuwj2H45sHGRa-N1PQ8qeBSzmM1ubx' 
!apt-get install unzip
!unzip 'competitive-data-science-predict-future-sales.zip' -d data
!gunzip 'data/test.csv.gz'
!gunzip 'data/sample_submission.csv.gz'
!gunzip 'data/sales_train.csv.gz'

--2019-05-10 07:08:49--  https://docs.google.com/uc?export=download&id=1-mpuwj2H45sHGRa-N1PQ8qeBSzmM1ubx
Resolving docs.google.com (docs.google.com)... 172.217.24.14, 2404:6800:4008:802::200e
Connecting to docs.google.com (docs.google.com)|172.217.24.14|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-ao-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ti7c4dluujdekv008ueb89g5tfi7dpr5/1557439200000/10974934039054397010/*/1-mpuwj2H45sHGRa-N1PQ8qeBSzmM1ubx?e=download [following]
--2019-05-10 07:08:52--  https://doc-00-ao-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ti7c4dluujdekv008ueb89g5tfi7dpr5/1557439200000/10974934039054397010/*/1-mpuwj2H45sHGRa-N1PQ8qeBSzmM1ubx?e=download
Resolving doc-00-ao-docs.googleusercontent.com (doc-00-ao-docs.googleusercontent.com)... 172.217.160.97, 2404:6800:4012:1::2001
Connecting to doc-00-ao-docs.googleusercontent.com (doc-00-ao-docs.googleuse

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import random
import datetime

In [2]:
# items = pd.read_csv('data/items.csv')
# shops = pd.read_csv('data/shops.csv')
# cats = pd.read_csv('data/item_categories.csv')
train = pd.read_csv('data/sales_train.csv', parse_dates=['date'], infer_datetime_format=True, dayfirst=True)

test  = pd.read_csv('data/test.csv').set_index('ID')

In [3]:
num_shops = train['shop_id'].max()
print ('number of shops: ',num_shops )

num_items = train['item_id'].max()
print ('number of items: ', num_items)
num_month = train['date_block_num'].max()
print ('number of month: ', num_month)
print ('size of train: ', train.shape)
train[:10]

number of shops:  59
number of items:  22169
number of month:  33
size of train:  (2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
5,2013-01-10,0,25,2564,349.00,1.0
6,2013-01-02,0,25,2565,549.00,1.0
7,2013-01-04,0,25,2572,239.00,1.0
8,2013-01-11,0,25,2572,299.00,1.0
9,2013-01-03,0,25,2573,299.00,3.0


In [4]:
test[:10]

,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268
5,5,5039
6,5,5041
7,5,5046
8,5,5319


In [5]:
np.array(test)

array([[    5,  5037],
       [    5,  5320],
       [    5,  5233],
       ...,
       [   45, 15757],
       [   45, 19648],
       [   45,   969]])

In [6]:
train_num = train.groupby(['date_block_num','shop_id','item_id']).sum().reset_index()
train_num = train_num.drop(labels=['item_price'], axis=1)
train_num[:10]


,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
5,0,0,61,1.0
6,0,0,75,1.0
7,0,0,88,1.0
8,0,0,95,1.0
9,0,0,96,1.0


In [7]:
def GetValue(data,inpu):
  data = np.array(data).astype('int')
  inpu = np.array(inpu).astype('int')
  
  lent = len(inpu)
  
  for i in range(lent):
    d = inpu[i]
    #print(d)
    data[d[1],d[2],d[0]] = d[3]
  return data
    

In [8]:
def CreatDataSe(train_num,num_shops,num_item,num_month):
  S = []
  for x in range(num_shops+1):
    I = []
    for y in range(num_item+1):
      M = []
      for z in range(num_month+1):
        value = 0
        M.append(value)
      I.append(M)
      #print('item : '+format(y))
    S.append(I)
    #print('shop : '+format(x))
  return np.array(S).astype('int')
  

In [9]:
S = CreatDataSe(train_num,num_shops,num_items,num_month)

In [10]:
S.shape

(60, 22170, 34)

In [11]:
S[58,10,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
G = GetValue(S,train_num)

In [13]:
G[10]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
np.array(test)

array([[    5,  5037],
       [    5,  5320],
       [    5,  5233],
       ...,
       [   45, 15757],
       [   45, 19648],
       [   45,   969]])

In [15]:
G.shape

(60, 22170, 34)

In [16]:
G[42,13485,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 2, 1, 0, 4, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
def create_dataset(dataset,on,look_back=2):
    dataX, dataY=[], []
    for i in range(len(dataset)-look_back):
        a=dataset[i:(i+look_back)]
        dataX.append(a)
        
        #dataX.append()
        dataY.append(dataset[i+look_back])
    return np.array(dataX), np.array(dataY)

In [18]:
def appendPrediction(NET,Data,week):#7 week
  for i in range(week):
    length  = len(Data)
    pre_in = np.array([[Data[length-2],Data[length-1]]])
    #print(length)
    
    pre_in = torch.unsqueeze(torch.from_numpy(pre_in), dim=1).float().cuda()
    #print(pre_in)
    peak_add = NET(pre_in).view(-1).data.cpu().numpy()[0]
    #print(peak_add)
    Data = np.append(Data,peak_add)
    
  return Data

In [19]:
class lstm_reg(torch.nn.Module):
  
  def __init__(self,input_size,hidden_size, output_size=1,num_layers=2):
    super(lstm_reg,self).__init__()

    self.rnn = torch.nn.LSTM(input_size,hidden_size,num_layers)
    self.reg = torch.nn.Linear(hidden_size,output_size)

  def forward(self,x):
    x, _ = self.rnn(x)
    s,b,h = x.shape
    x = x.view(s*b, h)
    x = self.reg(x)
    x = x.view(s,b,-1)
    return x

In [20]:
def trainAndPredictItem(data,a):
  a = np.array(a)
  data = np.array(data)
  iteamNum = []
  length = len(a)
  for i in range(length):
    d = data[a[i,0],a[i,1],:]
    #print(d)
    train_X2 ,train_Y2 = create_dataset(d.reshape(len(d),1),2)
    train_x2 = train_X2.reshape(-1,1,2)
    train_y2 = train_Y2.reshape(-1,1,1)
    train_x2 = Variable(torch.from_numpy(train_x2)).float()
    train_y2 = Variable(torch.from_numpy(train_y2)).float()
    Net = lstm_reg(2,8).cuda()
    criterion = torch.nn.MSELoss()
    optimizer1 = torch.optim.Adam(Net.parameters(),lr=1e-2)
    for e in range(10):
      var_x2 = Variable(train_x2).cuda()
      var_y2 = Variable(train_y2).cuda()
      out = Net(var_x2)
      loss = criterion(out, var_y2)
      optimizer1.zero_grad()
      loss.backward()
      optimizer1.step()
    pred = Net(Variable(train_x2).cuda())
    pred = pred.view(-1).data.cpu().numpy()
    pred = appendPrediction(Net,pred,1)
    s = pred[len(pred)-1]
    iteamNum.append(s)
    if i%800==0:
      print('Caculate: {}/{}, Finish:{:.2f} %'.format(i,length,100*(i/length)))
  return iteamNum

In [21]:
result = trainAndPredictItem(G,np.array(test))

Caculate: 0/214200, Finish:0.00 %
Caculate: 800/214200, Finish:0.37 %
Caculate: 1600/214200, Finish:0.75 %
Caculate: 2400/214200, Finish:1.12 %
Caculate: 3200/214200, Finish:1.49 %
Caculate: 4000/214200, Finish:1.87 %
Caculate: 4800/214200, Finish:2.24 %
Caculate: 5600/214200, Finish:2.61 %
Caculate: 6400/214200, Finish:2.99 %
Caculate: 7200/214200, Finish:3.36 %
Caculate: 8000/214200, Finish:3.73 %
Caculate: 8800/214200, Finish:4.11 %
Caculate: 9600/214200, Finish:4.48 %
Caculate: 10400/214200, Finish:4.86 %
Caculate: 11200/214200, Finish:5.23 %
Caculate: 12000/214200, Finish:5.60 %
Caculate: 12800/214200, Finish:5.98 %
Caculate: 13600/214200, Finish:6.35 %
Caculate: 14400/214200, Finish:6.72 %
Caculate: 15200/214200, Finish:7.10 %
Caculate: 16000/214200, Finish:7.47 %
Caculate: 16800/214200, Finish:7.84 %
Caculate: 17600/214200, Finish:8.22 %
Caculate: 18400/214200, Finish:8.59 %
Caculate: 19200/214200, Finish:8.96 %
Caculate: 20000/214200, Finish:9.34 %
Caculate: 20800/214200, Finis

Caculate: 168000/214200, Finish:78.43 %
Caculate: 168800/214200, Finish:78.80 %
Caculate: 169600/214200, Finish:79.18 %
Caculate: 170400/214200, Finish:79.55 %
Caculate: 171200/214200, Finish:79.93 %
Caculate: 172000/214200, Finish:80.30 %
Caculate: 172800/214200, Finish:80.67 %
Caculate: 173600/214200, Finish:81.05 %
Caculate: 174400/214200, Finish:81.42 %
Caculate: 175200/214200, Finish:81.79 %
Caculate: 176000/214200, Finish:82.17 %
Caculate: 176800/214200, Finish:82.54 %
Caculate: 177600/214200, Finish:82.91 %
Caculate: 178400/214200, Finish:83.29 %
Caculate: 179200/214200, Finish:83.66 %
Caculate: 180000/214200, Finish:84.03 %
Caculate: 180800/214200, Finish:84.41 %
Caculate: 181600/214200, Finish:84.78 %
Caculate: 182400/214200, Finish:85.15 %
Caculate: 183200/214200, Finish:85.53 %
Caculate: 184000/214200, Finish:85.90 %
Caculate: 184800/214200, Finish:86.27 %
Caculate: 185600/214200, Finish:86.65 %
Caculate: 186400/214200, Finish:87.02 %
Caculate: 187200/214200, Finish:87.39 %


In [32]:
save = np.array(result) + abs(np.min(np.array(result)))

In [33]:
save = save*0.35

In [34]:
dataset = pd.DataFrame({'item_cnt_month':save })

In [35]:
dataset.to_csv('submission.csv',header=True,index=True)